In [4]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import base64
import io
import psycopg2

import pandas as pd
import plotly.express as px

from moviepy.editor import VideoFileClip
import dash_table

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import plotly.express as px
from wordcloud import WordCloud
import plotly.graph_objects as go
from matplotlib import pyplot as plt
# plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False
plt.rc('font', family='NanumGothic')
plt.rc('font', family='NanumBarunGothic')
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import networkx as nx

In [8]:
pwd

'd:\\Github\\Genia\\Final_pjt2'

In [13]:
app = dash.Dash()

df = pd.read_csv('./data/Iris.csv')
fig = px.scatter(df, x="SepalWidth", y="SepalLength", color="Species")

with open('./Lecture_text/summarize/01강유리수의소수표현(1)_EBS중학뉴런수학2(상)_google_trans.txt', 'r', encoding='utf-8') as file:
    text = file.read()
# Preprocess the text data
nltk.download('punkt')
nltk.download('stopwords')
stop_words = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다', '.', ',', '(', ')', '!', '?', '-', '‘', '’', '“', '”', '…']
word_tokens = word_tokenize(text.lower())
filtered_text = [word for word in word_tokens if word.isalpha() and word not in stop_words]

# Create a bar plot of the most frequent words
word_counts = Counter(filtered_text)
most_common_words = word_counts.most_common(10)
nlp_fig1 = px.bar(x=[word[0] for word in most_common_words], y=[word[1] for word in most_common_words])

# Create a histogram of word length
word_lengths = [len(word) for word in filtered_text]
nlp_fig2 = px.histogram(x=word_lengths, nbins=max(word_lengths))

# Create a heatmap of term frequency
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(filtered_text)
term_frequency = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
nlp_fig3 = px.imshow(term_frequency)

# Create a network diagram
bigrams = nltk.bigrams(filtered_text)
bigram_freq = nltk.FreqDist(bigrams)
bigram_df = pd.DataFrame(bigram_freq.items(), columns=["bigram", "freq"])
bigram_df = bigram_df.sort_values(by="freq", ascending=False)

G = nx.Graph()

for index, row in bigram_df.iterrows():
    G.add_edge(row["bigram"][0], row["bigram"][1], weight=row["freq"])

pos = nx.spring_layout(G)
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))

node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append('# of connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text

nlp_fig4 = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                # title='Network graph made with Python',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                # annotations=[ dict(
                #     showarrow=False,
                #     xref="paper", yref="paper",
                #     x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )


app.layout = html.Div([
    # Title
    html.Div(
        children=[
            html.H1('Video Analysis',
                    style={'textAlign': 'center'}),
        ]),

    # Upload Video
    html.Div(
        dcc.Upload(
            id='upload-video',
            children=html.Div([
                'Drag and Drop or Select a Video',
            ]),
            style={
                'width': '100%',
                'height': '60px',
                'lineHeight': '60px',
                'borderWidth': '1px',
                'borderStyle': 'dashed',
                'borderRadius': '5px',
                'textAlign': 'center',
                'margin': '10px'
            },
            # Allow multiple files to be uploaded        
            multiple=True,
        ),
        style={'display': 'flex', 'justifyContent': 'center', 'alignItems': 'center'}
    ),

    # Output Video
    html.Div(id='output-video',
             style={'display': 'flex', 
                    'justifyContent': 'space-around', 
                    'alignItems': 'center'}),
    
    html.Br(),
    html.Br(),

    # feature 1
    html.Div(id='feature1',
             children=[
                 html.H2('Face Detection And Mesh'),
                 dcc.Graph(id='graph1', 
                           figure=fig)
             ]),
    html.Br(),

    # feature 2
    html.Div(id='feature2',
             children=[
                 html.H2('Sentimental Analysis'),
                 dcc.Graph(id='graph2', 
                           figure=fig)
             ]),   

    html.Br(),

    # feature 3
    html.Div(id='feature3',
             children=[
                 html.H2('Pose Detection'),
                 dcc.Graph(id='graph3', 
                           figure=fig)
             ]),  

    html.Br(),

    # feature 4
    html.Div(id='feature4',
             children=[
                 html.H2('Face & Arm Detection'),
                 dcc.Graph(id='graph4', 
                           figure=fig)
             ]),   

    html.Br(),

    # NLP_feature 1
    html.Div(id='NLP_feature 1',
             children=[
                 html.H2('The Most Frequent Words'),
                 dcc.Graph(id='NLP_graph1', 
                           figure=nlp_fig1)
             ]),   
    # NLP_feature 2
    html.Div(id='NLP_feature 2',
             children=[
                 html.H2('Histogram Of Word Length'),
                 dcc.Graph(id='NLP_graph2', 
                           figure=nlp_fig2)
             ]),   
    # NLP_feature 3
    html.Div(id='NLP_feature 3',
             children=[
                 html.H2('Heatmap Of Term Frequency'),
                 dcc.Graph(id='NLP_graph3', 
                           figure=nlp_fig3)
             ]),   
    # NLP_feature 4
    html.Div(id='NLP_feature 4',
             children=[
                 html.H2('Network Diagram'),
                 dcc.Graph(id='NLP_graph4', 
                           figure=nlp_fig4)
             ]),                                           

])


@app.callback(
    Output('output-video', 'children'),
    Input('upload-video', 'contents'),
    prevent_initial_call=True
)
def to_postgre(list_of_contents):
    if list_of_contents is not None:

        ########### About Database ###########
        # Connect to your postgres DB
        conn = psycopg2.connect("dbname=VideoWeb user=postgres password=1q2w3e")

        # Open a cursor to perform database operations
        cur = conn.cursor()

        # Create table if it doesn't exist
        cur.execute("CREATE TABLE IF NOT EXISTS video_metadata (id serial PRIMARY KEY, duration REAL, fps REAL, size TEXT, audio_fps REAL);")


        for contents in list_of_contents:
            # The contents are base64 encoded, so we must decode them
            decoded = base64.b64decode(contents.split(',')[1])

            # Write the binary data to a file
            with open('temp.mp4', 'wb') as f:
                f.write(decoded)

            # Read the video file and extract metadata
            clip = VideoFileClip('temp.mp4')

            # Extract metadata
            duration = clip.duration
            fps = clip.fps
            size = str(clip.size)
            audio_fps = clip.audio.fps if clip.audio is not None else None
            audio_nchannels = clip.audio.nchannels if clip.audio is not None else None
            audio_sample_width = clip.audio.sample_width if clip.audio is not None else None
            audio_sample_rate = clip.audio.sample_rate if clip.audio is not None else None

            # Insert the metadata into the database
            cur.execute("INSERT INTO video_metadata (duration, fps, size, audio_fps) VALUES (%s, %s, %s, %s);", (duration, fps, size, audio_fps))

        ########### About Table ###########
            # Create a DataFrame from the metadata
            df = pd.DataFrame({
                'Duration': [duration],
                'FPS': [fps],
                'Size': [size],
                'Audio FPS': [audio_fps],
                'Audio Channels': [audio_nchannels],
                'Audio Sample Width': [audio_sample_width],
                'Audio Sample Rate': [audio_sample_rate]
            
            })

            # Transpose the DataFrame
            df = df.transpose()
            df.columns = ['Value']            

            # Reset the index and rename the index column to 'Metadata'
            df = df.reset_index().rename(columns={'index': 'Metadata'})            

            # Create a DataTable from the DataFrame
            table = dash_table.DataTable(
                id='table',
                columns=[{"name": i, "id": i} for i in df.columns],
                data=df.to_dict('records'),
            )                  

        # Commit changes and close resources
        conn.commit()
        cur.close()
        conn.close()    

        children = [
            html.Div(
                children=[
                    html.Video(src=contents, 
                               controls=True, 
                               style={'width': '100%'})
                ],
                style={'width': '50%'}
            ),
            html.Div(
                children=[table],
                style={'width': '50%'}
            )
        ]
        return children
    


if __name__ == '__main__':
    app.run_server(debug=True)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hslio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hslio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
